In [32]:
from pycocotools.coco import COCO
import json
import pandas as pd
from PIL import Image
import shutil

In [4]:
dataDir='dataset/coco'
dataType='train2017'
annotationFile='dataset/coco/annotations/instances_train2017.json'

In [7]:
#loading coco train data annotations
coco = COCO(annotationFile)

loading annotations into memory...
Done (t=17.71s)
creating index...
index created!


In [8]:
classIDs = coco.getCatIds()# coco class ID
classes = coco.loadCats(classIDs) # list of dicts containting class info

In [11]:
len(set(classIDs))

80

In [14]:
# Reading the required classes (out of the 80).
with open('classes.txt') as f:
    content = f.readlines()
listOfClassesToFilterFromCOCO = [x.strip() for x in content]

# Fetch corresponding class IDs
imageIds = []
classIds = []
for className in listOfClassesToFilterFromCOCO: 
    classId = coco.getCatIds(catNms=className)
    classIds.append(classId[0])
    # Get all images containing the above class IDs
    imageIds.append(coco.getImgIds(catIds=classId))

#Flattening list
imageIds = [item for sublist in imageIds for item in sublist]
print("Number of images containing all the  classes:", len(imageIds))

Number of images containing all the  classes: 39318


In [15]:
#Getting unique imageIds
uniqueImageids = set(imageIds)

In [17]:
#Opening json file
with open(annotationFile) as f:
    instanceData = json.load(f)

In [21]:
#"bbox" format: [x,y,width,height]
#Filtering the data to get parameters only for required imageIds
imageID = []
categoryID = []
bbox= []
annotationsDict = instanceData['annotations']
#for uniqueImageid in uniqueImageids:
    #if any(d['image_id'] == 'uniqueImageid' for d in annotationsDict):
for i in range(len(annotationsDict)): 
    if instanceData['annotations'][i]['image_id'] in uniqueImageids and instanceData['annotations'][i]['category_id'] in classIds:
        imageID.append(instanceData['annotations'][i]['image_id'])
        categoryID.append(instanceData['annotations'][i]['category_id'])
        bbox.append(instanceData['annotations'][i]['bbox'])

In [24]:
#converting categoryID index from 0-80 to 0-8(classes.txt)
className = []
categoryID
for ID in categoryID:
    className.append(classes[next((index for (index, d) in enumerate(classes) if d["id"] == ID), None)]['name'])

classIndex = []
for i in range(len(className)):
    classIndex.append(listOfClassesToFilterFromCOCO.index(className[i]))

In [25]:
trainData = pd.DataFrame({'imageID': imageID, 'classIndex' : classIndex, 'bbox': bbox})

In [26]:
trainData['imageID'] = trainData['imageID'].astype('str')
trainData['imageID'] = trainData['imageID'].apply(lambda x: x.zfill(12))

In [28]:
#function to convert coco format (x, y, w, h) bbox  to yolo format scaled(x_center, y_center, w, h)
def convertBBOXToYolo(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

In [29]:
bboxInYOLO = []
for i, imageID in enumerate(list(trainData['imageID'])):
     img = Image.open(os.path.join(dataDir, dataType, imageID + '.jpg'))
     w = int(img.size[0])
     h = int(img.size[1])
     bboxInYOLO.append(convertBBOXToYolo((w,h),(bbox[i][0], bbox[i][0] + bbox[i][2], bbox[i][1], bbox[i][1] + bbox[i][3])))

In [30]:
trainData['bboxInYOLO'] = bboxInYOLO

In [31]:
trainData.head()

,imageID,classIndex,bbox,bboxInYOLO
0,000000275781,5,"[308.43, 35.25, 126.4, 108.71]","(0.74326, 0.23894666666666664, 0.2528000000000..."
1,000000403107,5,"[319.77, 9.19, 319.77, 286.26]","(0.7494609375, 0.3567213114754098, 0.499640625..."
2,000000203085,5,"[394.5, 42.74, 143.19, 143.19]","(0.7768250000000001, 0.27093601895734604, 0.23..."
3,000000520862,5,"[48.9, 27.33, 165.39, 149.57]","(0.27415625, 0.1595546875, 0.3445625, 0.233703..."
4,000000225913,5,"[516.13, 236.31, 63.95, 67.7]","(0.8564140625000001, 0.5628333333333333, 0.099..."


In [36]:
#creating label files for each image, copying filtered images to new directory
output_labels_path = os.path.join(dataDir, 'yolo/labels')
output_images_path = os.path.join(dataDir, 'yolo/images')
for out_path in [output_labels_path, output_images_path]:
    if not os.path.exists(out_path):
        os.makedirs(out_path)
for i in range(len(trainData)):
    with open(os.path.join(output_labels_path, str(trainData['imageID'][i]) + '.txt'), 'w') as f:
        f.write(str(trainData['classIndex'][i]) + " " + " ".join([str(a) for a in trainData['bboxInYOLO'][i]]) + '\n')
        shutil.copy(os.path.join(dataDir, dataType, str(trainData['imageID'][i]) + '.jpg'), output_images_path)
    